# Apache Parquet


![image.png](attachment:image.png)

Full Document

https://arrow.apache.org/docs/python/parquet.html

In [ ]:
import pyarrow.parquet as pq

In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa

In [ ]:
df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                   index=list('abc'))

In [ ]:
table = pa.Table.from_pandas(df)

In [ ]:
pq.write_table(table, 'example.parquet')

In [ ]:
table2 = pq.read_table('example.parquet')


In [ ]:
table2.to_pandas()

,one,two,three
a,-1.0,foo,True
b,NaN,bar,False
c,2.5,baz,True


In [ ]:
pq.read_table('example.parquet', columns=['one', 'three'])
 

pyarrow.Table
one: double
three: bool
----
one: [[-1,null,2.5]]
three: [[true,false,true]]

In [ ]:
pq.read_pandas('example.parquet', columns=['two']).to_pandas()

,two
a,foo
b,bar
c,baz


In [ ]:
#inspect meta data file
parquet_file = pq.ParquetFile('example.parquet')
parquet_file.metadata

  created_by: parquet-cpp-arrow version 8.0.0
  num_columns: 4
  num_rows: 3
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 2574

In [ ]:
metadata = pq.read_metadata('example.parquet')
metadata

  created_by: parquet-cpp-arrow version 8.0.0
  num_columns: 4
  num_rows: 3
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 2574

In [ ]:
parquet_file.num_row_groups

1

In [ ]:
parquet_file.read_row_group(0)

pyarrow.Table
one: double
two: string
three: bool
__index_level_0__: string
----
one: [[-1,null,2.5]]
two: [["foo","bar","baz"]]
three: [[true,false,true]]
__index_level_0__: [["a","b","c"]]

In [ ]:
metadata.row_group(0)

  num_columns: 4
  num_rows: 3
  total_byte_size: 282

We can similarly write a Parquet file with multiple row groups by using ParquetWriter:



In [ ]:
with pq.ParquetWriter('example2.parquet', table.schema) as writer:
   for i in range(3):
      writer.write_table(table)

In [ ]:
pf2 = pq.ParquetFile('example2.parquet')

In [ ]:
pf2.num_row_groups

3

In [ ]:
# data type handling
# specify data type
table3 = pq.read_table('example.parquet', filters= [ ("two", "in", {"bar"})], columns=['two'], 
                       read_dictionary=['stringb_c2'])

table3.to_pandas()

,two
0,bar


In [ ]:
#write to many partitions
# Local dataset write
pq.write_to_dataset(table, root_path='dataset_name',
                    partition_cols=['one', 'two'])

# Parquet and pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("PySpark Read Parquet").getOrCreate()

Sampledata =[("Ram ","","sharma","36636","M",4000), 
              ("Shyam ","Aggarwal","","40288","M",5000),
              ("Tushar ","","Garg","42114","M",5000),
              ("Sarita ","Kumar","Jain","39192","F",5000),
              ("Simran","Gupta","Brown","","F",-2)]

Samplecolumns=["firstname","middlename","lastname","dob","gender","salary"]
dataframe = spark.createDataFrame(Sampledata,Samplecolumns)
dataframe.write.mode("overwrite").parquet("./Samplepeople.parquet")

In [ ]:

ParDataFrame1 = spark.read.parquet("./Samplepeople.parquet")
ParDataFrame1.createOrReplaceTempView("ParquetTable")
ParDataFrame1.printSchema()
ParDataFrame1.show(truncate = False)


root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|dob  |gender|salary|
+---------+----------+--------+-----+------+------+
|Shyam    |Aggarwal  |        |40288|M     |5000  |
|Tushar   |          |Garg    |42114|M     |5000  |
|Ram      |          |sharma  |36636|M     |4000  |
|Sarita   |Kumar     |Jain    |39192|F     |5000  |
|Simran   |Gupta     |Brown   |     |F     |-2    |
+---------+----------+--------+-----+------+------+



In [ ]:
df = spark.read.csv('hdfs://localhost:9000/bank.csv', header = True, inferSchema = True)
df.printSchema()


root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [ ]:
#save to local parquet
df.write.mode('overwrite').parquet('./bank.parquet')
new_df = spark.read.parquet('./bank.parquet')
new_df.printSchema()


root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [ ]:
#write to hdfs

df.write.mode('overwrite').parquet('hdfs://localhost:9000/bank.parquet')
 
new_df = spark.read.parquet('hdfs://localhost:9000/bank.parquet' )
new_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)

